## container
container-test-beds/ml_keras_tf/ml_keras_tf.sif

In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
%load_ext autoreload
%autoreload 2

os.chdir('../Board_AI')

import boardai.config.config_board_AI as config
import boardai.nn.nn_utils as nn_utils


os.chdir('../unet')


from model import *
from data import *

import keras

import matplotlib, matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import cv2
from skimage import exposure
import random



In [ ]:
data_split_path = os.path.join(config.DATA_DIR_GENERAL, "segmentation_split")

train_dir =  os.path.join(data_split_path,  "train")
test_dir =  os.path.join(data_split_path,  "test")


img_folder = "imgs"
label_folder = "label"

train_files = glob.glob(train_dir + "/" + img_folder + "/*.png")
test_files = glob.glob(test_dir + "/" + img_folder + "/*.png")


In [ ]:
# import tensorflow as tf

# print(tf.__version__)

# print("\nis_gpu_available: ", tf.test.is_gpu_available(
#     cuda_only=False,
#     min_cuda_compute_capability=None))

# from tensorflow.python.client import device_lib
# print("\n",device_lib.list_local_devices())

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')


In [ ]:


# def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
#                     mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
#                     flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):

In [ ]:
batch_size = 20
# myGene = trainGenerator(3,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)
train_gen = trainGenerator(batch_size, train_dir,img_folder,label_folder,
                        data_gen_args,save_to_dir = None)
test_gen = trainGenerator(batch_size, test_dir,img_folder,label_folder,
                        data_gen_args,save_to_dir = None)


In [ ]:
img , mask = next(test_gen)

In [ ]:

plt.imshow(img[0,:,:,0] , cmap='gray')
plt.show()
plt.imshow(mask[0,:,:,0] , cmap='gray')

In [ ]:
print(np.unique(mask))
mask.shape

# Train

In [ ]:

model_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

model_path = "./logs/" + model_id

os.mkdir(model_path)


In [ ]:
model = unet()
metrics = []

history = model.fit_generator(
    train_gen,
    validation_steps=len(test_files) // batch_size,
    validation_data= test_gen, #(X_val, y_val),
    steps_per_epoch=2000 ,#len(train_files)// batch_size, # 2000,  # amount_training_samples // batch_size,
    epochs=5,#50,
    verbose=1,
    callbacks=[
        keras.callbacks.ModelCheckpoint(
            model_path + "/model.{epoch:02d}-{val_loss:.4f}.h5",
            save_best_only=True,
        ),
        keras.callbacks.EarlyStopping(
            monitor= "val_loss",
            patience=30,
        ),
        keras.callbacks.TensorBoard(
            log_dir=model_path,
            histogram_freq=0,
            write_graph=False,
            write_images=False,
        ),
#         keras.callbacks.ReduceLROnPlateau(
#             monitor=7,
#             factor=0.2,
#             patience=7,
#             min_lr=1e-4/ 10000,
#         ),
    ],
    use_multiprocessing=True,
    workers=4,
    # class_weight=hyperparams["loss_weights"],
)
metrics.append(history.history)


In [ ]:
df_metrics = pd.DataFrame(metrics[0])

In [ ]:
df_metrics.to_csv(
        os.path.join(model_path,  "losses.csv")
    )

In [ ]:
ax  = df_metrics.plot()
fig = ax.get_figure()


fig.savefig(
    os.path.join(model_path, "losses.png"), dpi=150
)

In [ ]:
 # Model to JSON
model_json = model.to_json()
with open(os.path.join(model_path, "model.json"), "w") as json_file:
    json_file.write(model_json)

In [ ]:
# END

### test  model and save predicted results

In [ ]:
#TODO: Define
model_path = 'logs/2020-10-27_17-13-35'

# test_set_name = 'pins-test__detectionDataset'  # 'DB3_testset'
test_set_name = 'DB3_testset'

In [ ]:
if(test_set_name == 'DB3_testset'):
    test_dir = os.path.join(config.DATA_DIR_GENERAL, "DB3_test_pins_task")

    test_files = glob.glob(os.path.join(test_dir, img_folder, '*.png'))


In [ ]:
test_path = os.path.join(test_dir,  img_folder)
storage_result_masks = os.path.join(model_path, "results", test_set_name)

In [ ]:
test_samples = len(test_files)
testGene = testGenerator(test_path, num_image=test_samples)#"data/membrane/test")
# model = unet()
# model.load_weights("unet_membrane.hdf5")
model = nn_utils.load_model(model_path)

results = model.predict_generator(testGene,test_samples,verbose=1)


if not os.path.exists(storage_result_masks):
    os.makedirs(storage_result_masks)

# saveResult("data/membrane/test",results)
saveResult(storage_result_masks,results, test_path)

# Evaluation metrics

In [ ]:
labels_path = os.path.join(test_dir,  label_folder)

testGene = testGenerator(test_path, labels_path=labels_path, num_image=test_samples)#"data/membrane/test")


In [ ]:
x_test = []
y_test = []

for i in range(test_samples):
    img, mks = next(testGene)
    
    x_test.append(img[0,:,:,:])
    y_test.append(mks[0,:,:,:])
    
x_test = np.array(x_test)
y_test = np.array(y_test)
    

In [ ]:
model = unet()
model_to_chose = nn_utils.select_model(model_path)
# load weights into new model
model.load_weights(os.path.join(model_path, model_to_chose))

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=20)
print("test loss, test acc:", results)

In [ ]:
plt.imshow(mks[0,:,:,0] , cmap='gray')


In [ ]:
np.unique(mks)

# results easy visualization

In [ ]:
sorage_vis_dir =  os.path.join(model_path,"res_visualization", test_set_name)

if not os.path.exists(sorage_vis_dir):
    os.makedirs(sorage_vis_dir)


In [ ]:
sample_size = 100

all_files = glob.glob(test_path + "/*.png")

sample_test_files = random.sample(all_files, sample_size)

In [ ]:
if not (test_set_name == 'DB3_testset'):
    
    for file_name in sample_test_files:
        sample_name =  file_name.split('/')[-1]

        label_path = labels_path + '/' + sample_name
        pred_path = storage_result_masks + '/' + sample_name


        img = cv2.imread(file_name, 0)
        label = cv2.imread(label_path, 0)
        pred = cv2.imread(pred_path, 0)

        img = exposure.adjust_gamma(img, config.GAMMA)


        #plotting
        fig,ax = plt.subplots(1, 3, figsize=(10,5))
        ax[0].imshow(img, cmap='gray')
        ax[1].imshow(label, cmap='gray')
        ax[2].imshow(pred, cmap='gray')
        ax[0].set(title="Original Img")
        ax[1].set(title="GT")
        ax[2].set(title="Prediciton")

        fig.tight_layout() # if needed
        fig.savefig( os.path.join(sorage_vis_dir, sample_name), dpi=100)


        plt.clf() 
        plt.close('all')   
        plt.close(fig)
else:
    #no segmentation GT for DB3
    for file_name in sample_test_files:
        sample_name =  file_name.split('/')[-1]

        pred_path = storage_result_masks + '/' + sample_name


        img = cv2.imread(file_name, 0)
        pred = cv2.imread(pred_path, 0)

        img = exposure.adjust_gamma(img, config.GAMMA)


        #plotting
        fig,ax = plt.subplots(1, 2, figsize=(10,5))
        ax[0].imshow(img, cmap='gray')
        ax[1].imshow(pred, cmap='gray')
        ax[0].set(title="Original Img")
        ax[1].set(title="Prediciton")

        fig.tight_layout() # if needed
        fig.savefig( os.path.join(sorage_vis_dir, sample_name), dpi=100)


        plt.clf() 
        plt.close('all')   
        plt.close(fig)